In [4]:
pip install  streamlit

  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.9 MB 2.1 MB/s eta 0:00:05
   ---- ----------------------------------- 1.0/9.9 MB

In [5]:
import pandas as pd
import requests
import unidecode

# ================================================================
# 1️⃣ CARGAR DATOS DESDE LA API (igual que en cargar_datos.py)
# ================================================================
def load_data_from_api(limit: int = 50000) -> pd.DataFrame:
    api_url = f"https://www.datos.gov.co/resource/nudc-7mev.json?$limit={limit}"
    response = requests.get(api_url)
    response.raise_for_status()
    data = response.json()
    df = pd.DataFrame(data)
    return df

df_raw = load_data_from_api(5000)  # prueba con 5000 para no saturar
print(f"✅ Datos cargados: {df_raw.shape}")
display(df_raw.head())

# ================================================================
# 2️⃣ FUNCIONES DE LIMPIEZA
# ================================================================
def normalizar_texto(texto: str) -> str:
    if pd.isna(texto):
        return texto
    return unidecode.unidecode(texto.strip().lower())

def corregir_departamentos(df: pd.DataFrame) -> pd.DataFrame:
    reemplazos = {
        "bogota d.c": "bogota",
        "bogotá d.c.": "bogota",
        "bogota": "bogota",
        "valle del cauca": "valle",
        "san andres, providencia y santa catalina": "san andres",
        "archipielago de san andres": "san andres"
    }
    df["departamento"] = df["departamento"].apply(lambda x: reemplazos.get(x, x))
    return df

def limpiar_metricas(df: pd.DataFrame) -> pd.DataFrame:
    for col in ["tasa_matriculaci_n_5_16", "cobertura_neta", "cobertura_bruta"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
            df.loc[df[col] < 0, col] = None
            df.loc[df[col] > 100, col] = 100
    return df

# ================================================================
# 3️⃣ LIMPIEZA Y NORMALIZACIÓN
# ================================================================
columnas_relevantes = [
    'a_o', 'departamento', 'municipio', 'c_digo_departamento',
    'poblaci_n_5_16', 'tasa_matriculaci_n_5_16',
    'cobertura_neta', 'cobertura_bruta'
]
df = df_raw[columnas_relevantes].dropna(how="all")

# Normalizar texto
for col in ["departamento", "municipio"]:
    df[col] = df[col].astype(str).apply(normalizar_texto)

df = corregir_departamentos(df)
df = limpiar_metricas(df)
df = df.drop_duplicates()

print("✅ Datos limpios:", df.shape)
display(df.head(10))

# ================================================================
# 4️⃣ DIMENSIONES Y TABLA DE HECHOS
# ================================================================
def crear_dimension(df, cols, nombre, sort_col=None):
    dim = df[cols].drop_duplicates()
    if sort_col:
        dim = dim.sort_values(by=sort_col)
    dim = dim.reset_index(drop=True)
    dim[f"id_{nombre}"] = dim.index + 1
    return dim[[f"id_{nombre}"] + cols]

dim_tiempo = crear_dimension(df, ['a_o'], 'tiempo', sort_col='a_o')
dim_geo = crear_dimension(df, ['c_digo_departamento', 'departamento', 'municipio'], 'geo', sort_col='c_digo_departamento')

print("Dim Tiempo:", dim_tiempo.shape)
display(dim_tiempo.head())

print("Dim Geografía:", dim_geo.shape)
display(dim_geo.head())

df_fact = df.merge(dim_tiempo, on='a_o') \
            .merge(dim_geo, on=['departamento', 'municipio', 'c_digo_departamento'], how='inner')

print("✅ Tabla de hechos:", df_fact.shape)
display(df_fact.head())


✅ Datos cargados: (5000, 39)


,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media
0,2023,11001,"Bogotá, D.C.",11,"Bogotá, D.C.",3766,"Bogotá, D.C. (ETC)",1141573,92.9,92.4,...,7.78,0.37,5.44,12.57,6.5,7.55,1.66,7.11,10.98,3.2
1,2023,19532,Patía,19,Cauca,3777,Cauca (ETC),7165,80.99,80.99,...,6.78,0.41,5.68,10.7,4.55,9.07,3.7,9.84,11.86,2.78
2,2023,47170,Chibolo,47,Magdalena,3794,Magdalena (ETC),5773,84.65,84.6,...,0.02,0,0,0,0.16,9.67,18.86,9.93,9.8,1.59
3,2023,68235,El Carmen de Chucurí,68,Santander,3808,Santander (ETC),4711,63.09,63.04,...,4.93,0.87,2.15,10.22,3.77,6.89,2.16,6.8,9.72,2.32
4,2023,63302,Génova,63,Quindio,3803,Quindio (ETC),1194,88.44,88.44,...,9.98,1.12,4.95,19.59,5.13,8.6,2.25,8.56,12.21,3.21


✅ Datos limpios: (5000, 8)


,a_o,departamento,municipio,c_digo_departamento,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_bruta
0,2023,"bogota, d.c.","bogota, d.c.",11,1141573,92.90,92.40,100.00
1,2023,cauca,patia,19,7165,80.99,80.99,93.20
2,2023,magdalena,chibolo,47,5773,84.65,84.60,100.00
3,2023,santander,el carmen de chucuri,68,4711,63.09,63.04,70.09
4,2023,quindio,genova,63,1194,88.44,88.44,99.58
5,2023,huila,la argentina,41,3302,84.65,84.65,95.46
6,2023,antioquia,peque,05,1900,75.32,75.32,83.11
7,2023,magdalena,cerro san antonio,47,2365,77.08,77.08,91.25
8,2023,cauca,almaguer,19,3445,60.58,60.58,75.85
9,2023,santander,guapota,68,458,75.98,75.98,84.72


Dim Tiempo: (5, 2)


,id_tiempo,a_o
0,1,2019
1,2,2020
2,3,2021
3,4,2022
4,5,2023


Dim Geografía: (1274, 4)


,id_geo,c_digo_departamento,departamento,municipio
0,1,0,nacional,nacional
1,2,05,antioquia,amalfi
2,3,05,antioquia,granada
3,4,05,antioquia,marinilla
4,5,05,antioquia,san pedro de uraba


✅ Tabla de hechos: (5000, 10)


,a_o,departamento,municipio,c_digo_departamento,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_bruta,id_tiempo,id_geo
0,2023,"bogota, d.c.","bogota, d.c.",11,1141573,92.90,92.40,100.00,5,151
1,2023,cauca,patia,19,7165,80.99,80.99,93.20,5,378
2,2023,magdalena,chibolo,47,5773,84.65,84.60,100.00,5,682
3,2023,santander,el carmen de chucuri,68,4711,63.09,63.04,70.09,5,1048
4,2023,quindio,genova,63,1194,88.44,88.44,99.58,5,948


In [6]:
# ==============================
# 🔍 1. Revisar BOGOTÁ antes de la limpieza
# ==============================
print("📌 Registros originales de Bogotá (tal como vienen de la API):")
bogota_original = df_raw[df_raw['departamento'].str.contains("bog", case=False, na=False)]
display(bogota_original[['departamento', 'municipio']].drop_duplicates().head(20))
print(f"Total registros Bogotá (sin limpiar): {len(bogota_original)}")

# ==============================
# 🧹 2. Revisar BOGOTÁ después de la limpieza
# (Ejecutar esto después de normalizar y corregir)
# ==============================
bogota_limpio = df[df['departamento'] == "bogota"]
print("\n✅ Registros de Bogotá después de la limpieza y normalización:")
display(bogota_limpio[['departamento', 'municipio']].drop_duplicates().head(20))
print(f"Total registros Bogotá (limpios): {len(bogota_limpio)}")


📌 Registros originales de Bogotá (tal como vienen de la API):


,departamento,municipio
0,"Bogotá, D.C.","Bogotá, D.C."
3772,Bogotá D.C.,Bogotá D.C.


Total registros Bogotá (sin limpiar): 5

✅ Registros de Bogotá después de la limpieza y normalización:


,departamento,municipio


Total registros Bogotá (limpios): 0


In [8]:
import pandas as pd
import requests
import unidecode

# ================================================================
# 1️⃣ CARGAR DATOS DESDE LA API
# ================================================================
def load_data_from_api(limit: int = 50000) -> pd.DataFrame:
    api_url = f"https://www.datos.gov.co/resource/nudc-7mev.json?$limit={limit}"
    response = requests.get(api_url)
    response.raise_for_status()
    data = response.json()
    df = pd.DataFrame(data)
    return df

# Para pruebas: cargamos pocos registros
df_raw = load_data_from_api(5000)
print(f"✅ Datos cargados: {df_raw.shape}")

# ================================================================
# 2️⃣ FUNCIONES DE LIMPIEZA
# ================================================================
def normalizar_texto(texto: str) -> str:
    """Minúsculas, sin tildes, sin espacios extra y sin puntos/comas."""
    if pd.isna(texto):
        return texto
    txt = unidecode.unidecode(texto.strip().lower())
    txt = txt.replace(".", "").replace(",", "")
    return txt

def corregir_departamentos(df: pd.DataFrame) -> pd.DataFrame:
    """
    Corrección automática de nombres problemáticos usando patrones.
    """
    df["departamento"] = df["departamento"].astype(str)
    df["departamento"] = df["departamento"].str.replace(r"bogota.*", "bogota", regex=True)
    df["departamento"] = df["departamento"].str.replace(r"san andres.*", "san andres", regex=True)
    df["departamento"] = df["departamento"].str.replace(r"valle del cauca", "valle", regex=True)
    return df

def limpiar_metricas(df: pd.DataFrame) -> pd.DataFrame:
    """Controla valores atípicos en métricas: 0-100% y numéricos."""
    for col in ["tasa_matriculaci_n_5_16", "cobertura_neta", "cobertura_bruta"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
            df.loc[df[col] < 0, col] = None
            df.loc[df[col] > 100, col] = 100
    return df

# ================================================================
# 3️⃣ LIMPIEZA Y NORMALIZACIÓN
# ================================================================
columnas_relevantes = [
    'a_o', 'departamento', 'municipio', 'c_digo_departamento',
    'poblaci_n_5_16', 'tasa_matriculaci_n_5_16',
    'cobertura_neta', 'cobertura_bruta'
]
df = df_raw[columnas_relevantes].dropna(how="all")

# Normalizar texto en departamento y municipio
for col in ["departamento", "municipio"]:
    df[col] = df[col].astype(str).apply(normalizar_texto)

# Corrección de nombres problemáticos
df = corregir_departamentos(df)

# Limpieza de métricas y duplicados
df = limpiar_metricas(df)
df = df.drop_duplicates()

print("✅ Datos limpios:", df.shape)
display(df.head(10))

# ================================================================
# 🔍 PRUEBA ESPECÍFICA: BOGOTÁ
# ================================================================
bogota_limpio = df[df['departamento'] == "bogota"]
print("\n✅ Registros de Bogotá después de la limpieza y normalización:")
display(bogota_limpio[['departamento', 'municipio']].drop_duplicates().head(20))
print(f"Total registros Bogotá (limpios): {len(bogota_limpio)}")

# ================================================================
# 4️⃣ DIMENSIONES Y TABLA DE HECHOS
# ================================================================
def crear_dimension(df, cols, nombre, sort_col=None):
    dim = df[cols].drop_duplicates()
    if sort_col:
        dim = dim.sort_values(by=sort_col)
    dim = dim.reset_index(drop=True)
    dim[f"id_{nombre}"] = dim.index + 1
    return dim[[f"id_{nombre}"] + cols]

dim_tiempo = crear_dimension(df, ['a_o'], 'tiempo', sort_col='a_o')
dim_geo = crear_dimension(df, ['c_digo_departamento', 'departamento', 'municipio'], 'geo', sort_col='c_digo_departamento')

print("Dim Tiempo:", dim_tiempo.shape)
display(dim_tiempo.head())

print("Dim Geografía:", dim_geo.shape)
display(dim_geo.head())

df_fact = df.merge(dim_tiempo, on='a_o') \
            .merge(dim_geo, on=['departamento', 'municipio', 'c_digo_departamento'], how='inner')

print("✅ Tabla de hechos:", df_fact.shape)
display(df_fact.head())


✅ Datos cargados: (5000, 39)
✅ Datos limpios: (5000, 8)


,a_o,departamento,municipio,c_digo_departamento,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_bruta
0,2023,bogota,bogota dc,11,1141573,92.90,92.40,100.00
1,2023,cauca,patia,19,7165,80.99,80.99,93.20
2,2023,magdalena,chibolo,47,5773,84.65,84.60,100.00
3,2023,santander,el carmen de chucuri,68,4711,63.09,63.04,70.09
4,2023,quindio,genova,63,1194,88.44,88.44,99.58
5,2023,huila,la argentina,41,3302,84.65,84.65,95.46
6,2023,antioquia,peque,05,1900,75.32,75.32,83.11
7,2023,magdalena,cerro san antonio,47,2365,77.08,77.08,91.25
8,2023,cauca,almaguer,19,3445,60.58,60.58,75.85
9,2023,santander,guapota,68,458,75.98,75.98,84.72



✅ Registros de Bogotá después de la limpieza y normalización:


,departamento,municipio
0,bogota,bogota dc


Total registros Bogotá (limpios): 5
Dim Tiempo: (5, 2)


,id_tiempo,a_o
0,1,2019
1,2,2020
2,3,2021
3,4,2022
4,5,2023


Dim Geografía: (1271, 4)


,id_geo,c_digo_departamento,departamento,municipio
0,1,0,nacional,nacional
1,2,05,antioquia,caceres
2,3,05,antioquia,hispania
3,4,05,antioquia,bello
4,5,05,antioquia,sabaneta


✅ Tabla de hechos: (5000, 10)


,a_o,departamento,municipio,c_digo_departamento,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_bruta,id_tiempo,id_geo
0,2023,bogota,bogota dc,11,1141573,92.90,92.40,100.00,5,150
1,2023,cauca,patia,19,7165,80.99,80.99,93.20,5,403
2,2023,magdalena,chibolo,47,5773,84.65,84.60,100.00,5,671
3,2023,santander,el carmen de chucuri,68,4711,63.09,63.04,70.09,5,1020
4,2023,quindio,genova,63,1194,88.44,88.44,99.58,5,951


# codigo original trnasformaciones 

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import io

# Colores oficiales Universidad Santo Tomás
UST_BLUE = "#002855"
UST_YELLOW = "#FFD100"
UST_GRAY = "#F5F5F5"
UST_WHITE = "#FFFFFF"

# Estilo general
st.markdown(f"""
    <style>
    .main {{
        background-color: {UST_GRAY};
    }}
    .stApp {{
        background-color: {UST_WHITE};
        color: #000000;
        font-family: 'Segoe UI', sans-serif;
    }}
    .stButton>button {{
        background-color: {UST_YELLOW};
        color: black;
        font-weight: bold;
        border-radius: 10px;
        padding: 0.5em 1em;
    }}
    .stDownloadButton>button {{
        background-color: {UST_BLUE};
        color: white;
        font-weight: bold;
        border-radius: 10px;
        padding: 0.5em 1em;
    }}
    .stTabs [data-baseweb="tab"] {{
        font-weight: bold;
        background-color: {UST_WHITE};
        color: {UST_BLUE};
        border-radius: 6px 6px 0 0;
        border: 1px solid #CCC;
    }}
    </style>
""", unsafe_allow_html=True)

def show_transform_tab():
    st.title("📊 Dashboard Educativo: Modelo Estrella")

    if 'df_raw' not in st.session_state:
        st.warning("🔺 Primero debes cargar los datos desde la pestaña correspondiente.")
        return

    df = st.session_state['df_raw'].copy()

    tabla_deptos = (
        df
        .query("departamento != 'NACIONAL'")
        [['c_digo_departamento','departamento']]
        .drop_duplicates()
        .groupby('c_digo_departamento')
        .sample(n=1, random_state=1)
        .reset_index()
        .drop(columns= 'index')
    )

    df = (
        df
        .query("departamento != 'NACIONAL'")
        .drop(columns = 'departamento')
        .merge(tabla_deptos, on = 'c_digo_departamento', how = 'left')
    )

    st.markdown("""
    ### 🛠️ Etapas del Flujo de Trabajo
    1. **Limpieza de datos**
    2. **Construcción de dimensiones**
    3. **Modelo estrella y tabla de hechos**
    4. **Visualización y métricas clave**
    5. **Descarga y resumen detallado**
    """)

    st.markdown("---")
    st.subheader("1️⃣ Limpieza y Validación de Datos")
    columnas_relevantes = [
        'a_o', 'departamento', 'municipio', 'c_digo_departamento',
        'poblaci_n_5_16', 'tasa_matriculaci_n_5_16',
        'cobertura_neta', 'cobertura_bruta'
    ]
    columnas_faltantes = [col for col in columnas_relevantes if col not in df.columns] # list comprehension
    if columnas_faltantes:
        st.error(f"❌ Columnas faltantes: {columnas_faltantes}")
        return
    df = df[columnas_relevantes]
    df.columns = [c.lower() for c in df.columns]
    for col in df.columns:
        if col not in ['departamento', 'municipio', 'c_digo_departamento']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df_clean = df.dropna()

    col1, col2 = st.columns(2)
    col1.metric("Registros originales", len(st.session_state['df_raw']))
    col2.metric("Registros válidos", len(df_clean))

    st.markdown("---")
    st.subheader("2️⃣ Dimensiones del Modelo Estrella")

    def crear_dimension(df, cols, nombre, sort_col=None):
        dim = df[cols].drop_duplicates()
        if sort_col:
            dim = dim.sort_values(by=sort_col)
        dim = dim.reset_index(drop=True)
        dim[f"id_{nombre}"] = dim.index + 1
        return dim[[f"id_{nombre}"] + cols]

    dim_tiempo = crear_dimension(df_clean, ['a_o'], 'tiempo')
    dim_geo = df_clean[['c_digo_departamento', 'departamento', 'municipio']].copy()
    dim_geo = dim_geo.sort_values(by=['c_digo_departamento', 'municipio'])
    dim_geo = dim_geo.drop_duplicates(subset=['c_digo_departamento'], keep='first').reset_index(drop=True)
    dim_geo['id_geo'] = dim_geo.index + 1
    dim_geo = dim_geo[['id_geo', 'c_digo_departamento', 'departamento', 'municipio']]

    col3, col4 = st.columns(2)
    col3.metric("Dimensión Tiempo", len(dim_tiempo))
    col4.metric("Dimensión Geográfica", len(dim_geo))

    st.markdown("---")
    st.subheader("3️⃣ Tabla de Hechos")

    df_fact = df_clean.merge(dim_tiempo, on='a_o') \
                      .merge(dim_geo, on=['departamento', 'municipio', 'c_digo_departamento'], how='inner')

    df_fact = df_fact[[
        'id_tiempo', 'id_geo',
        'poblaci_n_5_16', 'tasa_matriculaci_n_5_16',
        'cobertura_neta', 'cobertura_bruta']]

    st.success(f"✅ Tabla de hechos construida con {len(df_fact):,} registros.")
    st.session_state['df_fact'] = df_fact
    st.session_state['dim_geo'] = dim_geo
    st.session_state['dim_tiempo'] = dim_tiempo

    st.markdown("---")
    st.subheader("4️⃣ Indicadores y Visualizaciones")

    escolaridad_prom = df_fact.groupby('id_geo')[['tasa_matriculaci_n_5_16']].mean().reset_index()
    escolaridad_prom = escolaridad_prom.merge(dim_geo, on='id_geo')
    top_mpios = escolaridad_prom.sort_values(by='tasa_matriculaci_n_5_16', ascending=False).head(10)

    fig = px.bar(
        top_mpios,
        x='municipio',
        y='tasa_matriculaci_n_5_16',
        title='Top 10 Municipios con Mayor Tasa de Escolaridad (5-16 años)',
        labels={'tasa_matriculaci_n_5_16': 'Tasa de Escolaridad (%)'},
        color_discrete_sequence=[UST_BLUE]
    )
    st.plotly_chart(fig, use_container_width=True)

    cobertura_depto = df_fact.merge(dim_geo, on='id_geo') \
        .groupby('departamento')['cobertura_neta'].mean().sort_values(ascending=False).head(10)
    st.markdown("**🏛️ Top Departamentos por Cobertura Neta Promedio**")
    st.dataframe(cobertura_depto.reset_index())

    st.markdown("---")
    st.subheader("5️⃣ Vista y Descarga de la Tabla de Hechos")

    st.dataframe(df_fact.head(50))
    output = io.BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        df_fact.to_excel(writer, index=False, sheet_name='TablaHechos')
    output.seek(0)

    st.download_button(
        label="📥 Descargar Tabla de Hechos",
        data=output,
        file_name='tabla_hechos_educacion.xlsx',
        mime='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

    st.markdown("---")
    st.subheader("📈 Resumen por Departamento y Año")

    df_fact_ext = df_fact.merge(dim_geo, on='id_geo').merge(dim_tiempo, on='id_tiempo')
    resumen = df_fact_ext.groupby(['departamento', 'a_o'])[
        ['tasa_matriculaci_n_5_16', 'cobertura_neta', 'cobertura_bruta']].mean().reset_index()
    st.dataframe(resumen.head(20))


# codigo 3 

In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
import unidecode
import io

# Colores oficiales Universidad Santo Tomás
UST_BLUE = "#002855"
UST_YELLOW = "#FFD100"
UST_GRAY = "#F5F5F5"
UST_WHITE = "#FFFFFF"

# Estilo general
st.markdown(f"""
    <style>
    .main {{
        background-color: {UST_GRAY};
    }}
    .stApp {{
        background-color: {UST_WHITE};
        color: #000000;
        font-family: 'Segoe UI', sans-serif;
    }}
    .stButton>button {{
        background-color: {UST_YELLOW};
        color: black;
        font-weight: bold;
        border-radius: 10px;
        padding: 0.5em 1em;
    }}
    .stDownloadButton>button {{
        background-color: {UST_BLUE};
        color: white;
        font-weight: bold;
        border-radius: 10px;
        padding: 0.5em 1em;
    }}
    .stTabs [data-baseweb="tab"] {{
        font-weight: bold;
        background-color: {UST_WHITE};
        color: {UST_BLUE};
        border-radius: 6px 6px 0 0;
        border: 1px solid #CCC;
    }}
    </style>
""", unsafe_allow_html=True)

# =========================
# 🔧 Funciones de Limpieza
# =========================
def normalizar_texto(texto: str) -> str:
    if pd.isna(texto):
        return texto
    txt = unidecode.unidecode(texto.strip().lower())
    txt = txt.replace(".", "").replace(",", "")
    return txt

def corregir_departamentos(df: pd.DataFrame) -> pd.DataFrame:
    df["departamento"] = df["departamento"].astype(str)
    df["departamento"] = df["departamento"].str.replace(r"bogota.*", "bogota", regex=True)
    df["departamento"] = df["departamento"].str.replace(r"san andres.*", "san andres", regex=True)
    df["departamento"] = df["departamento"].str.replace(r"valle del cauca", "valle", regex=True)
    return df

def limpiar_metricas(df: pd.DataFrame) -> pd.DataFrame:
    for col in ["tasa_matriculaci_n_5_16", "cobertura_neta", "cobertura_bruta"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
            df.loc[df[col] < 0, col] = None
            df.loc[df[col] > 100, col] = 100
    return df

# =========================
# 🖥️ Dashboard Principal
# =========================
def show_transform_tab():
    st.title("📊 Dashboard Educativo: Modelo Estrella")

    if 'df_raw' not in st.session_state:
        st.warning("🔺 Primero debes cargar los datos desde la pestaña correspondiente.")
        return

    df = st.session_state['df_raw'].copy()

    # --- Limpieza inicial de NACIONAL ---
    tabla_deptos = (
        df.query("departamento != 'NACIONAL'")
          [['c_digo_departamento', 'departamento']]
          .drop_duplicates()
          .groupby('c_digo_departamento')
          .sample(n=1, random_state=1)
          .reset_index()
          .drop(columns='index')
    )

    df = (
        df.query("departamento != 'NACIONAL'")
          .drop(columns='departamento')
          .merge(tabla_deptos, on='c_digo_departamento', how='left')
    )

    st.markdown("""
    ### 🛠️ Etapas del Flujo de Trabajo
    1. **Limpieza de datos**
    2. **Construcción de dimensiones**
    3. **Modelo estrella y tabla de hechos**
    4. **Visualización y métricas clave**
    5. **Descarga y resumen detallado**
    """)

    # =========================
    # 1️⃣ Limpieza y Validación
    # =========================
    st.markdown("---")
    st.subheader("1️⃣ Limpieza y Validación de Datos")

    columnas_relevantes = [
        'a_o', 'departamento', 'municipio', 'c_digo_departamento',
        'poblaci_n_5_16', 'tasa_matriculaci_n_5_16',
        'cobertura_neta', 'cobertura_bruta'
    ]
    columnas_faltantes = [col for col in columnas_relevantes if col not in df.columns]
    if columnas_faltantes:
        st.error(f"❌ Columnas faltantes: {columnas_faltantes}")
        return

    df = df[columnas_relevantes]

    # --- Normalización de texto ---
    for col in ["departamento", "municipio"]:
        df[col] = df[col].astype(str).apply(normalizar_texto)

    df = corregir_departamentos(df)
    df = limpiar_metricas(df)
    df = df.drop_duplicates()
    df_clean = df.dropna()

    col1, col2 = st.columns(2)
    col1.metric("Registros originales", len(st.session_state['df_raw']))
    col2.metric("Registros válidos", len(df_clean))
    st.session_state['df_clean'] = df_clean

    # =========================
    # 2️⃣ Dimensiones
    # =========================
    st.markdown("---")
    st.subheader("2️⃣ Dimensiones del Modelo Estrella")

    def crear_dimension(df, cols, nombre, sort_col=None):
        dim = df[cols].drop_duplicates()
        if sort_col:
            dim = dim.sort_values(by=sort_col)
        dim = dim.reset_index(drop=True)
        dim[f"id_{nombre}"] = dim.index + 1
        return dim[[f"id_{nombre}"] + cols]

    dim_tiempo = crear_dimension(df_clean, ['a_o'], 'tiempo', sort_col='a_o')
    dim_geo = crear_dimension(df_clean, ['c_digo_departamento', 'departamento', 'municipio'],
                              'geo', sort_col='c_digo_departamento')

    col3, col4 = st.columns(2)
    col3.metric("Dimensión Tiempo", len(dim_tiempo))
    col4.metric("Dimensión Geográfica", len(dim_geo))

    st.session_state['dim_geo'] = dim_geo
    st.session_state['dim_tiempo'] = dim_tiempo

    # =========================
    # 3️⃣ Tabla de Hechos
    # =========================
    st.markdown("---")
    st.subheader("3️⃣ Tabla de Hechos")

    df_fact = df_clean.merge(dim_tiempo, on='a_o') \
                      .merge(dim_geo, on=['departamento', 'municipio', 'c_digo_departamento'], how='inner')
    df_fact = df_fact[[
        'id_tiempo', 'id_geo',
        'poblaci_n_5_16', 'tasa_matriculaci_n_5_16',
        'cobertura_neta', 'cobertura_bruta'
    ]]

    st.success(f"✅ Tabla de hechos construida con {len(df_fact):,} registros.")
    st.session_state['df_fact'] = df_fact

    # =========================
    # 4️⃣ Indicadores y Visualizaciones
    # =========================
    st.markdown("---")
    st.subheader("4️⃣ Indicadores y Visualizaciones")

    escolaridad_prom = df_fact.groupby('id_geo')[['tasa_matriculaci_n_5_16']].mean().reset_index()
    escolaridad_prom = escolaridad_prom.merge(dim_geo, on='id_geo')
    top_mpios = escolaridad_prom.sort_values(by='tasa_matriculaci_n_5_16', ascending=False).head(10)

    fig = px.bar(
        top_mpios,
        x='municipio',
        y='tasa_matriculaci_n_5_16',
        title='Top 10 Municipios con Mayor Tasa de Escolaridad (5-16 años)',
        labels={'tasa_matriculaci_n_5_16': 'Tasa de Escolaridad (%)'},
        color_discrete_sequence=[UST_BLUE]
    )
    st.plotly_chart(fig, use_container_width=True)

    cobertura_depto = df_fact.merge(dim_geo, on='id_geo') \
        .groupby('departamento')['cobertura_neta'].mean().sort_values(ascending=False).head(10)
    st.markdown("**🏛️ Top Departamentos por Cobertura Neta Promedio**")
    st.dataframe(cobertura_depto.reset_index())

    # =========================
    # 5️⃣ Descarga y Resumen
    # =========================
    st.markdown("---")
    st.subheader("5️⃣ Vista y Descarga de la Tabla de Hechos")

    st.dataframe(df_fact.head(50))
    output = io.BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        df_fact.to_excel(writer, index=False, sheet_name='TablaHechos')
    output.seek(0)

    st.download_button(
        label="📥 Descargar Tabla de Hechos",
        data=output,
        file_name='tabla_hechos_educacion.xlsx',
        mime='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
    )

    st.markdown("---")
    st.subheader("📈 Resumen por Departamento y Año")

    df_fact_ext = df_fact.merge(dim_geo, on='id_geo').merge(dim_tiempo, on='id_tiempo')
    resumen = df_fact_ext.groupby(['departamento', 'a_o'])[
        ['tasa_matriculaci_n_5_16', 'cobertura_neta', 'cobertura_bruta']].mean().reset_index()
    st.dataframe(resumen.head(20))


ModuleNotFoundError: No module named 'streamlit'